In [1]:
import numpy as np
import pandas as pd
import ripser
import matplotlib.pyplot as plt
from persim.persistent_entropy import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.tree import DecisionTreeClassifier

from sklearn import datasets

Concept:
1. find 10 neighbors in each row
2. calculate entropy
3. the result is the featur for the row

Problem
1. the entropy is not the global !

In [2]:
iris = datasets.load_iris()
X = pd.DataFrame(iris.data, columns = iris.feature_names)
y = iris['target']

In [3]:
nbrs = NearestNeighbors(n_neighbors=20).fit(X)
entropy_feature = []
for i in range(X.shape[0]):

    distances, indices = nbrs.kneighbors(X.iloc[[i]])
    row_neighbor = X.iloc[indices[0][1:]]

    dgm = ripser.ripser(row_neighbor)['dgms'][0]
    p_entropy = persistent_entropy(dgm)
    entropy_feature.append(p_entropy)

In [4]:
X['ktopo_feature'] = [x.tolist()[0] if isinstance(x, np.ndarray) else x for x in entropy_feature]
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),ktopo_feature
0,5.1,3.5,1.4,0.2,2.838358
1,4.9,3.0,1.4,0.2,2.850062
2,4.7,3.2,1.3,0.2,2.856186
3,4.6,3.1,1.5,0.2,2.859755
4,5.0,3.6,1.4,0.2,2.836136


In [5]:
X_final = X[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)', 'ktopo_feature']].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled,y_train)

training_prediction = log_reg.predict(X_train_scaled)
test_prediction = log_reg.predict(X_test_scaled)

print("Precision, Recall, Confusion matrix, in training\n")
print(metrics.classification_report(y_train, training_prediction, digits=3))
print(metrics.confusion_matrix(y_train, training_prediction))

Precision, Recall, Confusion matrix, in training

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        21
           1      0.960     0.889     0.923        27
           2      0.897     0.963     0.929        27

    accuracy                          0.947        75
   macro avg      0.952     0.951     0.951        75
weighted avg      0.948     0.947     0.947        75

[[21  0  0]
 [ 0 24  3]
 [ 0  1 26]]


In [7]:
entropy_feature = []
for i in range(X.shape[0]):

    data_remove = X.drop(index=i)
    dgm = ripser.ripser(data_remove)['dgms'][0]
    p_entropy = persistent_entropy(dgm)
    entropy_feature.append(p_entropy)

In [8]:
X['topo_feature'] = [x.tolist()[0] if isinstance(x, np.ndarray) else x for x in entropy_feature]
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),ktopo_feature,topo_feature
0,5.1,3.5,1.4,0.2,2.838358,4.872607
1,4.9,3.0,1.4,0.2,2.850062,4.871961
2,4.7,3.2,1.3,0.2,2.856186,4.870529
3,4.6,3.1,1.5,0.2,2.859755,4.871125
4,5.0,3.6,1.4,0.2,2.836136,4.871865


In [9]:
X_final = X.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.5, random_state=42)

In [10]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
X_train_scaled[0]

array([-0.54270972,  0.01713978,  0.34804919,  0.29802754, -1.85957316,
       -0.29157254])

In [12]:
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled,y_train)

training_prediction = log_reg.predict(X_train_scaled)
test_prediction = log_reg.predict(X_test_scaled)

print("Precision, Recall, Confusion matrix, in training\n")
print(metrics.classification_report(y_train, training_prediction, digits=3))
print(metrics.confusion_matrix(y_train, training_prediction))

Precision, Recall, Confusion matrix, in training

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        21
           1      0.962     0.926     0.943        27
           2      0.929     0.963     0.945        27

    accuracy                          0.960        75
   macro avg      0.963     0.963     0.963        75
weighted avg      0.960     0.960     0.960        75

[[21  0  0]
 [ 0 25  2]
 [ 0  1 26]]


In [13]:
print(metrics.accuracy_score(y_train, training_prediction))

0.96


In [16]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train_scaled, y_train)
dtree.score(X_test_scaled, y_test)

0.9466666666666667